In [1]:
from pycaret.classification import *
import pandas as pd
import os
import wandb

os.chdir('/home/antoine/gene_pheno_pred')
os.environ["CUDA_VISIBLE_DEVICES"]="1"
os.environ["WANDB_API_KEY"]="4e5748d6c6f3917c78cdc38a516a1bac776faf58"
wandb.login()
wandb.init(
    project="cigap-classif",
)


wandb: Currently logged in as: antoine-toffano (mlb). Use `wandb login --relogin` to force relogin


In [2]:
# Load training set
df1 = pd.read_csv('/home/antoine/gene_pheno_pred/2023-05-17_18-04-22_test.csv' , header=0)
df2 = pd.read_csv('/home/antoine/gene_pheno_pred/2023-05-17_18-04-22_val.csv' , header=0)
df_val = pd.concat([df1, df2], axis=0)
df = pd.read_csv('/home/antoine/gene_pheno_pred/2023-05-17_18-04-22_train.csv' , header=0)

df['link'] = df['ground_truth'].apply(lambda x: 1 if x != 'no_link_known' else 0)
df_val['link'] = df_val['ground_truth'].apply(lambda x: 1 if x != 'no_link_known' else 0)

df

,0,1,2,3,4,5,6,7,8,9,...,192,193,194,195,196,197,198,199,ground_truth,link
0,-0.039281,0.045129,0.060632,0.040718,0.019589,-0.037644,-0.046621,0.035634,-0.036399,0.040565,...,0.048025,0.031555,-0.031849,-0.014078,0.053716,-0.064128,-0.030465,-0.024128,no_link_known,0
1,-0.036916,0.028429,0.049426,0.027237,-0.035879,-0.043270,-0.047764,0.030260,-0.019672,0.043986,...,0.053235,0.036724,0.048768,0.003633,-0.035456,-0.047571,0.012309,-0.046372,no_link_known,0
2,-0.045044,-0.040876,0.053277,-0.023103,-0.040964,-0.057454,-0.043918,0.036199,-0.010271,0.061459,...,0.051839,0.048165,-0.043459,-0.050535,-0.030699,-0.040411,-0.003266,-0.021388,no_link_known,0
3,-0.185021,0.170344,-0.216889,-0.203051,-0.165906,0.221980,0.152171,0.172068,-0.212461,-0.209708,...,-0.215327,-0.212621,-0.205381,-0.211254,0.199636,-0.207717,-0.214102,0.211480,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,1
4,-0.037551,0.039643,0.025630,-0.039976,0.007117,-0.040056,-0.035566,0.048091,-0.042153,0.026733,...,-0.004194,-0.028268,0.029635,-0.042946,0.049204,0.029149,-0.030395,-0.041650,no_link_known,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2364529,-0.058883,0.048679,0.058004,-0.076264,0.003280,-0.079589,-0.073878,0.028624,-0.017588,0.070026,...,-0.247503,-0.229071,0.187853,-0.163784,-0.202127,-0.029653,0.194864,0.070248,http://www.semanticweb.org/needed-terms#001,1
2364530,-0.003803,0.026271,0.029825,-0.006390,0.043357,0.036364,-0.028140,-0.048597,0.013899,0.040898,...,-0.018761,-0.035790,0.027299,-0.024690,0.038224,0.013004,-0.033128,-0.023349,no_link_known,0
2364531,-0.029159,0.038472,0.016023,0.034138,0.018736,-0.034391,-0.038524,0.025481,-0.044181,0.040296,...,0.008625,0.026602,-0.020006,0.005575,0.005215,-0.019102,0.017794,0.019887,no_link_known,0
2364532,0.010675,0.045282,0.048201,0.019762,0.023624,-0.031469,-0.040448,0.036446,-0.038341,0.036148,...,0.121935,0.125016,0.305975,-0.270114,0.237756,0.039959,0.260827,0.262957,no_link_known,0


In [3]:
df2 = df
df3 = df_val

In [4]:
df = df.drop('ground_truth', axis=1)
df_val = df_val.drop('ground_truth', axis=1)

In [5]:
# Experiment setup
s = setup(df, target = 'link', n_jobs=-1, system_log=True, use_gpu = True, log_experiment='wandb')
exp = ClassificationExperiment()

[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recomp

,Description,Value
0,Session id,1545
1,Target,link
2,Target type,Binary
3,Original data shape,"(2364534, 201)"
4,Transformed data shape,"(2364534, 201)"
5,Transformed train set shape,"(1655173, 201)"
6,Transformed test set shape,"(709361, 201)"
7,Numeric features,200
8,Preprocess,True
9,Imputation type,simple


[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDA=1


# Random Forest

In [6]:
rf = create_model('rf')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9853,0.9993,0.9981,0.9733,0.9855,0.9707,0.9710
1,0.9858,0.9993,0.9983,0.9739,0.9860,0.9716,0.9719
2,0.9854,0.9993,0.9981,0.9733,0.9855,0.9707,0.9710
3,0.9859,0.9993,0.9983,0.9741,0.9860,0.9717,0.9720
4,0.9857,0.9993,0.9981,0.9739,0.9858,0.9713,0.9716
5,0.9859,0.9993,0.9980,0.9744,0.9861,0.9719,0.9722
6,0.9860,0.9993,0.9984,0.9742,0.9861,0.9720,0.9723
7,0.9862,0.9993,0.9982,0.9748,0.9863,0.9724,0.9726
8,0.9855,0.9992,0.9981,0.9736,0.9857,0.9710,0.9713


In [7]:
rf_pred = predict_model(rf, data=df_val)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Random Forest Classifier,0.5268,0.5684,0.1516,0.6072,0.2426,0.0535,0.0810


In [8]:
save_model(rf, 'models_classif/complex/binary_emb_train/rf')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=FastMemory(location=/tmp/joblib),
          steps=[('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['0', '1', '2', '3', '4', '5', '6',
                                              '7', '8', '9', '10', '11', '12',
                                              '13', '14', '15', '16', '17', '18',
                                              '19', '20', '21', '22', '23', '24',
                                              '25', '26', '27', '28', '29', ...],
                                     transformer=SimpleImputer(add_indicator=False,
                                                               copy=True,
                                                               fill_value=None,
                                                               keep_empty_feat...
                  RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                         class_weight=None, c

# Extra Trees

In [9]:
et = create_model('et')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9845,0.9992,0.9980,0.9717,0.9847,0.9690,0.9693
1,0.9848,0.9992,0.9980,0.9723,0.9850,0.9696,0.9699
2,0.9847,0.9992,0.9981,0.9720,0.9849,0.9694,0.9697
3,0.9848,0.9991,0.9978,0.9725,0.9850,0.9695,0.9699
4,0.9844,0.9992,0.9977,0.9719,0.9846,0.9689,0.9692
5,0.9847,0.9991,0.9977,0.9724,0.9849,0.9694,0.9697
6,0.9849,0.9992,0.9979,0.9726,0.9851,0.9698,0.9701
7,0.9852,0.9991,0.9978,0.9732,0.9853,0.9703,0.9706
8,0.9845,0.9990,0.9978,0.9719,0.9847,0.9689,0.9693


In [10]:
et_pred = predict_model(et, data=df_val)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Extra Trees Classifier,0.5255,0.5765,0.1507,0.6018,0.2411,0.0510,0.0770


In [11]:
save_model(et, 'models_classif/complex/binary_emb_train/et')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=FastMemory(location=/tmp/joblib),
          steps=[('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['0', '1', '2', '3', '4', '5', '6',
                                              '7', '8', '9', '10', '11', '12',
                                              '13', '14', '15', '16', '17', '18',
                                              '19', '20', '21', '22', '23', '24',
                                              '25', '26', '27', '28', '29', ...],
                                     transformer=SimpleImputer(add_indicator=False,
                                                               copy=True,
                                                               fill_value=None,
                                                               keep_empty_feat...
                  ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0,
                                       class_weight=None, crit

# Light Gradient Boosting Machine

In [12]:
lightgbm = create_model('lightgbm')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9836,0.9982,0.9971,0.9708,0.9838,0.9671,0.9675
1,0.9833,0.9981,0.9971,0.9704,0.9836,0.9667,0.9670
2,0.9831,0.9982,0.9970,0.9700,0.9833,0.9661,0.9665
3,0.9838,0.9982,0.9968,0.9715,0.9840,0.9676,0.9679
4,0.9838,0.9982,0.9967,0.9716,0.9840,0.9676,0.9679
5,0.9833,0.9981,0.9970,0.9705,0.9835,0.9666,0.9670
6,0.9838,0.9981,0.9968,0.9714,0.9840,0.9675,0.9678
7,0.9840,0.9983,0.9966,0.9721,0.9842,0.9680,0.9683
8,0.9831,0.9979,0.9968,0.9703,0.9834,0.9663,0.9666


In [13]:
lgbm_pred = predict_model(lightgbm, data=df_val)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.5335,0.6083,0.2156,0.5920,0.3161,0.0671,0.0869


In [14]:
save_model(lightgbm, 'models_classif/complex/binary_emb_train/lightgbm')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=FastMemory(location=/tmp/joblib),
          steps=[('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['0', '1', '2', '3', '4', '5', '6',
                                              '7', '8', '9', '10', '11', '12',
                                              '13', '14', '15', '16', '17', '18',
                                              '19', '20', '21', '22', '23', '24',
                                              '25', '26', '27', '28', '29', ...],
                                     transformer=SimpleImputer(add_indicator=False,
                                                               copy=True,
                                                               fill_value=None,
                                                               keep_empty_feat...
                  LGBMClassifier(boosting_type='gbdt', class_weight=None,
                                 colsample_bytree=1.0, impo